In [1]:
#!pip install openai
#!pip install PyPDF2
#!pip install python-pptx

In [2]:
TOKEN = 'XXXXX'

In [3]:
import os
import datetime
import openai
import PyPDF2
from pptx import Presentation
openai.api_key = TOKEN
MAX_CHUNK_SIZE = 1000

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role":"user","content": "My name is Tim, can you act as a professional english writer and help me to refine my writing in a professional tone?"},
  ]
)
print(response["choices"][0]["message"]["content"])

Certainly, Tim! I'd be glad to help you refine your writing in a professional tone. Do you have a specific project or piece of writing you'd like me to look at?


In [ ]:
class bcolors: # color setup
  RED   = "\033[1;31m"
  GREEN = '\033[92m'
  BLUE = '\033[94m'

def print_w(s): #setup the print width
  width = 1000
  while len(s) > width:
    print(f'{s[:width]:<{width}}')
    s = s[width:]
  print(s)
  
def chatGPT_api(list_msg,list_ans,message): # API parameter setup
  # setup system role
  system_role = "You are a professional english writing assistant."  #@param {type:"string",title:""}   
  send_msg=[{"role":"system","content": system_role}]

  # Read historical conversations
  for i in range(len(list_msg)):
    _msg={"role":"user","content": list_msg[i]}
    send_msg.append(_msg)
    _ans={"role":"assistant","content": list_ans[i]}
    send_msg.append(_ans)

  # Prepare user's new message
  _msg={"role":"user","content": message}
  send_msg.append(_msg)

  # Call the API and return the result
  response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=send_msg
  )
  #return the results
  return response["choices"][0]["message"]["content"]

def main():
  # List of historical conversation records
  history_list_msg=[]
  history_list_ans=[]
  
  while(True):
    # waiting for user input
    print(f"👦 {bcolors.GREEN}", end="")
    message = input()
    # Check if user wants to exit the program
    if message.lower() == "exit":
        break
    
    # Check if user wants to reply to an email
    elif message.lower() == "/re":
        reply = input("My name is Tim, from the best strategy consulting firms, and I want to write about: ")
        email_context = input("Reply email context: ")
        try:
            message = f"Reply to this email {email_context}. Be polite and say {reply}"
            answer = chatGPT_api(history_list_msg, history_list_ans, message)
            print(f"⚛️ {bcolors.BLUE}")
            print_w(answer)

            history_list_msg.append(message)
            history_list_ans.append(answer)
        
        except Exception as e:
            #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
            # Clear chat history and continue
            history_list_msg.clear()
            history_list_ans.clear()
            continue
    
    # Check if user wants to write a new email
    elif message.lower() == "/new":
        email = input("My name is Tim, from the best strategy consulting firms, I want to write about: ")
        try:
            message = f"I need to write to an email, please provide the subject for this {email}, and rewrite {email}"
            answer = chatGPT_api(history_list_msg, history_list_ans, message)
            print(f"⚛️ {bcolors.BLUE}")
            print_w(answer)

            history_list_msg.append(message)
            history_list_ans.append(answer)
        
        except Exception as e:
            #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
            # Clear chat history and continue
            history_list_msg.clear()
            history_list_ans.clear()
            continue
            
    # Check if user wants to refine a writing
    elif message.lower() == "/rw":
        writing_txt = input("My name is Tim, from the best strategy consulting firms, I want to write about: ")
        try:
            message = f"Please help me to refine and rewrite {writing_txt}"
            answer = chatGPT_api(history_list_msg, history_list_ans, message)
            print(f"⚛️ {bcolors.BLUE}")
            print_w(answer)

            history_list_msg.append(message)
            history_list_ans.append(answer)
        except Exception as e:
            #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
            # Clear chat history and continue
            history_list_msg.clear()
            history_list_ans.clear()
            continue
        
    # Check if user wants to summarize a writing
    elif message.lower() == "/sum":
        writing_txt = input("the paragraph: ")
        try:
            message = f"My name is Tim, from the best strategy consulting firms, please help me to summarize {writing_txt}"
            answer = chatGPT_api(history_list_msg, history_list_ans, message)
            print(f"⚛️ {bcolors.BLUE}")
            print_w(answer)
            
            history_list_msg.append(message)
            history_list_ans.append(answer)
        except Exception as e:
            #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
            # Clear chat history and continue
            history_list_msg.clear()
            history_list_ans.clear()
            continue
            
    # Check if user wants to summarize a writing (big size)
    elif message.lower() == "/big":
        writing_txt = input("the paragraph: ")
        message = f"My name is Tim, from the best strategy consulting firms, please help me to summarize {writing_txt}"
        # Define the maximum chunk size
        max_chunk_size = MAX_CHUNK_SIZE

        # Break the input text into chunks
        chunks = [writing_txt[i:i+max_chunk_size] for i in range(0, len(writing_txt), max_chunk_size)]

        # Process each chunk separately
        for chunk in chunks:
            try:
                message = f"Please help me to summarize {chunk}"
                answer = chatGPT_api(history_list_msg, history_list_ans, message)
                print(f"{bcolors.BLUE}")
                print_w(answer)
                
                history_list_msg.append(message)
                history_list_ans.append(answer)
            except Exception as e:
                #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
                # Clear chat history and continue
                history_list_msg.clear()
                history_list_ans.clear()
                continue
    
    # Check if user wants to summarize a pdf file (big size)
    elif message.lower() == "/pdf":
        # prompt user to enter the file path of the folder
        folder_path = input("Enter the path of the PDF files: ")
        print(folder_path)
        
        # create an empty list to hold the extracted text
        text_list = []
        
        # Loop through all the files in the folder
        for filename in os.listdir(folder_path):
            # Check if the file is a PDF file
            if filename.endswith(".pdf"):
                # Open the PDF file in read binary mode
                with open(os.path.join(folder_path, filename), "rb") as pdf_file:
                    # Create a PDF reader object
                    pdf_reader = PyPDF2.PdfReader(pdf_file)

                    # loop through each page of the PDF file
                    for page_num in range(len(pdf_reader.pages)):
                        # get the page object
                        #page_obj = pdf_reader.getPage(page_num)
                        page_obj = pdf_reader.pages[page_num]
                        # extract the text from the page
                        page_text = page_obj.extract_text()
                        # append the text to the list
                        text_list.append(page_text)                  
        #print(text_list)
        # Get the current date and time
        now = datetime.datetime.now()
        # Format the date and time as a string
        timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
        # Set the file name using the timestamp
        filename = f"Raw_pdf_{timestamp}.txt"

        # Print the name of the saved file
        print(f"Data saved to {filename}.")

        # Open the file in write mode and save the data
        with open(filename, "w") as f:
            for item in text_list:
                try:
                    f.write(item + "\n")
                except Exception as e:
                    continue
                    
        # Combine all the text into a single string
        all_text = ' '.join(text_list)
    
        # Define the maximum chunk size
        max_chunk_size = MAX_CHUNK_SIZE

        # Break the input text into chunks
        chunks = [all_text[i:i+max_chunk_size] for i in range(0, len(all_text), max_chunk_size)]

        # Process each chunk separately and store the summaries in a list
        summaries = []
        
        # Process each chunk separately
        for chunk in chunks:
            try:
                message = f"Please help me to summarize {chunk}"
                answer = chatGPT_api(history_list_msg, history_list_ans, message)
                print(f"{bcolors.BLUE}")
                print_w(answer)
                summaries.append(answer)
                history_list_msg.append(message)
                history_list_ans.append(answer)
            except Exception as e:
                #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
                # Clear chat history and continue
                history_list_msg.clear()
                history_list_ans.clear()
                continue
    
        # Combine all the summaries into a single string
        final_summary = ' '.join(summaries)

        # Get the current date and time
        now = datetime.datetime.now()
        # Format the date and time as a string
        timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
        # Set the file name using the timestamp
        filename = f"Summary_PDF_response_{timestamp}.txt"

        # Open the file in write mode and save the data
        with open(filename, "w") as f:
            f.write(final_summary)

        # Print the name of the saved file
        print(f"Data saved to {filename}.")     
    
    # Check if user wants to summarize a ppt file (big size)
    elif message.lower() == "/ppt":
        # prompt user to enter the file path of the folder
        folder_path = input("Enter the path of the PPT files: ")
        print(folder_path)
        
        # create an empty list to hold the extracted text
        text_list = []

        # Loop through all the files in the folder
        for filename in os.listdir(folder_path):
            # Check if the file is a PPT file
            if filename.endswith('.pptx'):
                # Load the presentation
                prs = Presentation(os.path.join(folder_path, filename))
                            
                # Loop through all the slides in the presentation
                for slide in prs.slides:
                    # Create a new text variable for each slide
                    page_text = ''

                    # Loop through all the shapes in the slide
                    for shape in slide.shapes:
                        # If the shape contains text, add it to the slide text variable
                        if hasattr(shape, 'text'):
                            page_text += shape.text + " "
                    # append the text to the list
                    text_list.append(page_text)
                            

        # Get the current date and time
        now = datetime.datetime.now()
        # Format the date and time as a string
        timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
        # Set the file name using the timestamp
        filename = f"Raw_ppt_{timestamp}.txt"

        # Print the name of the saved file
        print(f"Data saved to {filename}.")

        # Open the file in write mode and save the data
        with open(filename, "w") as f:
            for item in text_list:
                f.write(item + "\n")
                
        # Combine all the text into a single string
        all_text = ' '.join(text_list)
    
        # Define the maximum chunk size
        max_chunk_size = MAX_CHUNK_SIZE

        # Break the input text into chunks
        chunks = [all_text[i:i+max_chunk_size] for i in range(0, len(all_text), max_chunk_size)]

        # Process each chunk separately and store the summaries in a list
        summaries = []
        
        # Process each chunk separately
        for chunk in chunks:
            try:
                message = f"Please help me to summarize {chunk}"
                answer = chatGPT_api(history_list_msg, history_list_ans, message)
                print(f"{bcolors.BLUE}")
                print_w(answer)
                summaries.append(answer)
                history_list_msg.append(message)
                history_list_ans.append(answer)
            except Exception as e:
                #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
                # Clear chat history and continue
                history_list_msg.clear()
                history_list_ans.clear()
                continue
    
        # Combine all the summaries into a single string
        final_summary = ' '.join(summaries)

        # Get the current date and time
        now = datetime.datetime.now()
        # Format the date and time as a string
        timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
        # Set the file name using the timestamp
        filename = f"Summary_PPT_response_{timestamp}.txt"

        # Open the file in write mode and save the data
        with open(filename, "w") as f:
            f.write(final_summary)

        # Print the name of the saved file
        print(f"Data saved to {filename}.")   
                
    # Check if user wants to summarize a txt file (big size)
    elif message.lower() == "/txt":
        # prompt user to enter the file path of the folder
        folder_path = input("Enter the path of the txt files: ")
        print(folder_path)
        
        # create an empty list to hold the extracted text
        text_list = []
        
        # Loop through all the files in the folder    
        for filename in os.listdir(folder_path):
            # Check if the file is a txt file
            if filename.endswith('.txt'):
                # Load the presentation
                with open(os.path.join(folder_path, filename), 'r') as file:
                    # read the contents of the file
                    contents = file.read()
                    # append the text to the list
                    text_list.append(contents)                   
        
        # Combine all the text into a single string
        all_text = ' '.join(text_list)
    
        # Define the maximum chunk size
        max_chunk_size = MAX_CHUNK_SIZE

        # Break the input text into chunks
        chunks = [all_text[i:i+max_chunk_size] for i in range(0, len(all_text), max_chunk_size)]

        # Process each chunk separately and store the summaries in a list
        summaries = []
        
        # Process each chunk separately
        for chunk in chunks:
            try:
                message = f"Please help me to summarize {chunk}"
                answer = chatGPT_api(history_list_msg, history_list_ans, message)
                print(f"{bcolors.BLUE}")
                print_w(answer)
                summaries.append(answer)
                history_list_msg.append(message)
                history_list_ans.append(answer)
            except Exception as e:
                #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
                # Clear chat history and continue
                history_list_msg.clear()
                history_list_ans.clear()
                continue
    
        # Combine all the summaries into a single string
        final_summary = ' '.join(summaries)

        # Get the current date and time
        now = datetime.datetime.now()
        # Format the date and time as a string
        timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")
        # Set the file name using the timestamp
        filename = f"Summary_TXT_response_{timestamp}.txt"

        # Open the file in write mode and save the data
        with open(filename, "w") as f:
            f.write(final_summary)

        # Print the name of the saved file
        print(f"Data saved to {filename}.")
    
     # Check if user wants to translate a writing
    elif message.lower() == "/tr":
        input_lang = input("Input languages: ")
        translate_lang = input("Translate to languages: ")
        writing_txt = input("the paragraph: ")
        message = f"Please help me to translate {input_lang} to {translate_lang} --- the paragraph {writing_txt}"
        # Define the maximum chunk size
        max_chunk_size = MAX_CHUNK_SIZE

        # Break the input text into chunks
        chunks = [writing_txt[i:i+max_chunk_size] for i in range(0, len(writing_txt), max_chunk_size)]

        # Process each chunk separately
        for chunk in chunks:
            try:
                message = f"Please help me to translate {input_lang} to {translate_lang} --- the paragraph {chunk}"
                answer = chatGPT_api(history_list_msg, history_list_ans, message)
                print(f"{bcolors.BLUE}")
                print_w(answer)

                history_list_msg.append(message)
                history_list_ans.append(answer)
            except Exception as e:
                #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
                # Clear chat history and continue
                history_list_msg.clear()
                history_list_ans.clear()
                continue
                
    else:
    # Call the API and return the result
        try:
            answer = chatGPT_api(history_list_msg, history_list_ans, message)
            print(f"⚛️ {bcolors.BLUE}")
            print_w(answer)

            history_list_msg.append(message)
            history_list_ans.append(answer)
        
        except Exception as e:
            #print(f"{bcolors.RED} Error summarizing {filename}: {str(e)}")
            # Clear chat history and continue
            history_list_msg.clear()
            history_list_ans.clear()
            continue

if __name__ == "__main__":
  main()

👦 /re
My name is Tim, from the best strategy consulting firms, and I want to write about: Got it. We can do it next time. Just FYI, I don't mind to join a call between 12AM - 1AM, I have another European client workshop next week starts at 12AM. As long as there is no conflict with other client workshop and it starts before 1AM PST. I can make it.
Reply email context: Thanks so much Tim.  Which would be super late for you.
⚛️ 
Thank you for letting me know. Got it, we can do it next time. 

I am used to working at that time. As long as there are no conflicts with other client workshops and the call starts before 1AM PST, I can make it work.
👦 /re
My name is Tim, from the best strategy consulting firms, and I want to write about: I understand, I will wait for next time to talk to them. 
Reply email context: Which would be super late for you. 
⚛️ 
Thank you for letting me know. I completely understand that it could be super late for me. I appreciate your consideration. 

I am happy to wa